In [2]:
##
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import math
import os

In [3]:
oldcard1=pd.read_csv(r'F:\扬州\oldcard1_11.csv',)
oldcard1

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期
0,0,9616900720343197,6,20220611184643,1.0,160,16,104643,20220611
1,1,9616900720229892,6,20220611184819,1.0,160,16,104819,20220611
2,2,9616900722020003,6,20220611184837,1.0,160,16,104837,20220611
3,3,9616900721987090,101,20220611201457,1.6,160,16,121457,20220611
4,4,9616900711049089,101,20220611203704,0.0,160,16,123704,20220611
...,...,...,...,...,...,...,...,...,...
46663,46663,9616900722268015,101,20220612014737,1.6,270,27,934737,20220611
46664,46664,9616900722196402,101,20220612015350,1.6,270,27,935350,20220611
46665,46665,9616900720382304,101,20220612015539,1.6,270,27,935539,20220611
46666,46666,9616900722246864,6,20220612020146,0.0,270,27,940146,20220611


In [9]:
oldcard1.sort_values(by='时间')

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期
1162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611
1866,1866,9616900721991115,6,20220611130132,1.0,880,88,50132,20220611
1867,1867,9616900730081595,6,20220611130529,1.0,880,88,50529,20220611
44791,44791,9616900720346426,11,20220611130550,0.0,6270,627,50550,20220611
14314,14314,9616900722242310,6,20220611131021,1.0,880,88,51021,20220611
...,...,...,...,...,...,...,...,...,...
1618,1618,9616900101114963,101,20220612061548,1.6,551,55,981548,20220611
21404,21404,9616900722011730,11,20220612061601,0.0,180,18,981601,20220611
30231,30231,9616900720110302,101,20220612061700,1.6,200,20,981700,20220611
25259,25259,9616900722227009,6,20220612062146,1.0,661,66,982146,20220611


In [4]:
#线路索引
IDSY = oldcard1.drop_duplicates(subset=['线路编号'],keep='first')
IDSY.sort_values(by='线路编号',inplace=True)
IDSY.reset_index(drop=True,inplace=True) 
IDSY

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611
1,2209,9616900720307617,6,20220611141041,1.0,20,2,61041,20220611
2,9428,9616900722227198,6,20220611153036,0.0,30,3,73036,20220611
3,1241,0,11,20220611142155,0.0,50,5,62155,20220611
4,41758,0,11,20220611150926,0.0,60,6,70926,20220611
...,...,...,...,...,...,...,...,...,...
159,8173,9616900730130399,6,20220611162433,0.5,3850,385,82433,20220611
160,4351,9616900730079653,11,20220611145849,0.0,3870,387,65849,20220611
161,43,9616900730044189,11,20220611142552,0.0,6120,612,62552,20220611
162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611


In [69]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框

IDSY['段开始时间']=None
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0


t1=40000+0*10000
t2=40000+0*10000+3000
#t2=i*100000+6000
##筛选时间段内的所有刷卡数据
listt=[]   
for g in range(0,len(oldcard1)):
    TIME=int(oldcard1['时间'].iloc[g])
    if TIME>=t1 and TIME<=t2:
        listt.append(g)
oldcard2=oldcard1.loc[listt,:]
oldcard2.reset_index(drop=True,inplace=True) 
print('oldcard2')
print(oldcard2)
##按照线路名统计每条线上的刷卡数据
for i in range(0,len(IDSY)):
    IDSY['段开始时间'].iloc[i]=t1
    tsum=0
    list2=[]
    for k in range(0,len(oldcard2)):
        id1=oldcard2['线路编号'].iloc[k]
        id2=IDSY['线路编号'].iloc[i] 
        
        if id1==id2:                  
            tsum = tsum+1
            IDSY['总客运量'].iloc[i]=tsum
            list2.append(k)
    oldcard3=oldcard2.loc[list2,:]
    #print('oldcard3')
    #print(oldcard3)
    ##线路相同的情况下的刷卡类型统计
    oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
    #print('oldcard4')
    #print(oldcard4)
    osum=len(oldcard4)
    IDSY['老年卡客运量'].iloc[i]=osum
    fosum=tsum-osum
    IDSY['其他卡客运量'].iloc[i]=fosum
ALLIDSY=IDSY       
ALLIDSY

oldcard2
Empty DataFrame
Columns: [Unnamed: 0, 卡号, 卡类型, 交易时间需减去8小时, 交易金额, 线路号, 线路编号, 时间, 日期]
Index: []


,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611,40000,0,0,0
1,2209,9616900720307617,6,20220611141041,1.0,20,2,61041,20220611,40000,0,0,0
2,9428,9616900722227198,6,20220611153036,0.0,30,3,73036,20220611,40000,0,0,0
3,1241,0,11,20220611142155,0.0,50,5,62155,20220611,40000,0,0,0
4,41758,0,11,20220611150926,0.0,60,6,70926,20220611,40000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,8173,9616900730130399,6,20220611162433,0.5,3850,385,82433,20220611,40000,0,0,0
160,4351,9616900730079653,11,20220611145849,0.0,3870,387,65849,20220611,40000,0,0,0
161,43,9616900730044189,11,20220611142552,0.0,6120,612,62552,20220611,40000,0,0,0
162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611,40000,0,0,0


In [70]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
for i in range(1,12):
    t1=40000+i*10000
    t2=40000+i*10000+3000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    ##按照线路名统计每条线上的刷卡数据
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t1
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
      ##  print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY=ALLIDSY.append(IDSY)      
ALLIDSY

oldcard2
    Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0         1163  9616900721791451  101  20220611131034   2.4  6220   622   
1         1610  9616900721753777  101  20220611131126   1.6   710    71   
2         1611  9616900720344603    6  20220611132126   1.0   710    71   
3         1866  9616900721991115    6  20220611130132   1.0   880    88   
4         1867  9616900730081595    6  20220611130529   1.0   880    88   
5         1868  9616900722147137    6  20220611131522   1.0   880    88   
6         1869  9616900722207916    6  20220611131528   1.0   880    88   
7         8636  9616900720229560    6  20220611132625   1.0   880    88   
8         8637  9616900722096778    6  20220611132804   1.0   880    88   
9        10096  9616900722020389    6  20220611132111   1.0   150    15   
10       10097  9616900721708619    6  20220611132146   1.0   150    15   
11       10098  9616900722187269    6  20220611132254   1.0   150    15   
12       10099  

oldcard2
      Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0             25                 0   11  20220611180001   0.0   380    38   
1             26  9616900722005833   11  20220611181705   0.0   380    38   
2             48  9616900730054972  101  20220611180151   2.4  6120   612   
3            160  9616900720195975   11  20220611182816   0.0  1020   102   
4            268  9616900721797212    6  20220611180410   1.0   160    16   
...          ...               ...  ...             ...   ...   ...   ...   
1910       46502  9616900722011776   11  20220611180550   0.0   780    78   
1911       46503  9616900760030932  101  20220611180553   0.0   780    78   
1912       46504  9616900722124671  101  20220611180558   0.0   780    78   
1913       46534  9616900722024710   11  20220611182931   0.0   380    38   
1914       46535  9616900721600966   11  20220611182935   0.0   380    38   

          时间        日期  
0     100001  20220611  
1     101705  20

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611,50000,0,0,0
1,2209,9616900720307617,6,20220611141041,1.0,20,2,61041,20220611,50000,0,0,0
2,9428,9616900722227198,6,20220611153036,0.0,30,3,73036,20220611,50000,0,0,0
3,1241,0,11,20220611142155,0.0,50,5,62155,20220611,50000,0,0,0
4,41758,0,11,20220611150926,0.0,60,6,70926,20220611,50000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,8173,9616900730130399,6,20220611162433,0.5,3850,385,82433,20220611,150000,0,0,0
160,4351,9616900730079653,11,20220611145849,0.0,3870,387,65849,20220611,150000,0,0,0
161,43,9616900730044189,11,20220611142552,0.0,6120,612,62552,20220611,150000,6,3,9
162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611,150000,8,3,11


In [71]:
ALLIDSY3=ALLIDSY
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
IDSY['段开始时间']=0
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0
for i in range(0,8):
    t1=920000+i*10000
    t2=920000+i*10000+3000
    t3=40000+(i+12)*10000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    ##按照线路名统计每条线上的刷卡数据
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t3
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
        #print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY3=ALLIDSY3.append(IDSY)      
ALLIDSY3

oldcard2
      Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0             14  9616900721576893  101  20220612000159   1.6   160    16   
1             53  9616900730127920    6  20220612000641   0.0  6120   612   
2             54  9616900730046951    6  20220612001724   0.0  6120   612   
3            267  9616900722259800    6  20220612002048   0.0  1010   101   
4            331  9616900720235844   10  20220612002247   0.0   160    16   
...          ...               ...  ...             ...   ...   ...   ...   
1814       46593  9616900722124520    6  20220612000135   1.0   880    88   
1815       46594  9616900730031468   10  20220612000626   0.0   880    88   
1816       46595                 0   10  20220612001441   0.0   880    88   
1817       46596  9616900721610966  101  20220612002507   0.0   880    88   
1818       46657  9616900730068322   11  20220612002208   0.0  2030   203   

          时间        日期  
0     920159  20220611  
1     920641  20

oldcard2
Empty DataFrame
Columns: [Unnamed: 0, 卡号, 卡类型, 交易时间需减去8小时, 交易金额, 线路号, 线路编号, 时间, 日期]
Index: []


,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611,50000,0,0,0
1,2209,9616900720307617,6,20220611141041,1.0,20,2,61041,20220611,50000,0,0,0
2,9428,9616900722227198,6,20220611153036,0.0,30,3,73036,20220611,50000,0,0,0
3,1241,0,11,20220611142155,0.0,50,5,62155,20220611,50000,0,0,0
4,41758,0,11,20220611150926,0.0,60,6,70926,20220611,50000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,8173,9616900730130399,6,20220611162433,0.5,3850,385,82433,20220611,230000,0,0,0
160,4351,9616900730079653,11,20220611145849,0.0,3870,387,65849,20220611,230000,0,0,0
161,43,9616900730044189,11,20220611142552,0.0,6120,612,62552,20220611,230000,0,0,0
162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611,230000,0,0,0


In [72]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
IDSY['段开始时间']=None
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0
#t1=40000+0*10000
t1=40000+0*10000+3000
t2=40000+0*10000+6000
#筛选时间段内的所有刷卡数据
listt=[]   
for g in range(0,len(oldcard1)):
    TIME=int(oldcard1['时间'].iloc[g])
    if TIME>=t1 and TIME<=t2:
        listt.append(g)
oldcard2=oldcard1.loc[listt,:]
oldcard2.reset_index(drop=True,inplace=True) 
print('oldcard2')
print(oldcard2)
##按照线路名统计每条线上的刷卡数据
for i in range(0,len(IDSY)):
    IDSY['段开始时间'].iloc[i]=t1
    tsum=0
    list2=[]
    for k in range(0,len(oldcard2)):
        id1=oldcard2['线路编号'].iloc[k]
        id2=IDSY['线路编号'].iloc[i] 
        if id1==id2:                  
            tsum = tsum+1
            IDSY['总客运量'].iloc[i]=tsum
            list2.append(k)
    oldcard3=oldcard2.loc[list2,:]
    #print('oldcard3')
    #print(oldcard3)
    ##线路相同的情况下的刷卡类型统计
    oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
    #print('oldcard4')
    #print(oldcard4)
    osum=len(oldcard4)
    IDSY['老年卡客运量'].iloc[i]=osum
    fosum=tsum-osum
    IDSY['其他卡客运量'].iloc[i]=fosum
ALLIDSY2=IDSY       
ALLIDSY2

oldcard2
   Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号     时间  \
0        1162  9616900730121574    6  20220611124737   1.5  6220   622  44737   

         日期  
0  20220611  


,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611,43000,0,0,0
1,2209,9616900720307617,6,20220611141041,1.0,20,2,61041,20220611,43000,0,0,0
2,9428,9616900722227198,6,20220611153036,0.0,30,3,73036,20220611,43000,0,0,0
3,1241,0,11,20220611142155,0.0,50,5,62155,20220611,43000,0,0,0
4,41758,0,11,20220611150926,0.0,60,6,70926,20220611,43000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,8173,9616900730130399,6,20220611162433,0.5,3850,385,82433,20220611,43000,0,0,0
160,4351,9616900730079653,11,20220611145849,0.0,3870,387,65849,20220611,43000,0,0,0
161,43,9616900730044189,11,20220611142552,0.0,6120,612,62552,20220611,43000,0,0,0
162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611,43000,1,0,1


In [73]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
for i in range(1,12):
    t1=40000+i*10000+3000
    t2=40000+i*10000+6000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    ##按照线路名统计每条线上的刷卡数据
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t1
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
      ##  print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY2=ALLIDSY2.append(IDSY)      
ALLIDSY2

oldcard2
     Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0          1164  9616900722002852   11  20220611133526   0.0  6220   622   
1          1165  9616900730049706    6  20220611133537   1.5  6220   622   
2          1569  9616900721765570    6  20220611134939   1.0  2320   232   
3          1570  9616900730013087    6  20220611135412   1.0  2320   232   
4          1571  9616900730046950    6  20220611135416   1.0  2320   232   
..          ...               ...  ...             ...   ...   ...   ...   
277       45469  9616900721558271  101  20220611135924   2.4  6120   612   
278       45871  9616900722140035   11  20220611135059   0.0   870    87   
279       46386  9616900721960321   11  20220611134902   0.0   780    78   
280       46387  9616900720111694  101  20220611134906   1.6   780    78   
281       46388  9616900720039562  101  20220611134957   1.6   780    78   

        时间        日期  
0    53526  20220611  
1    53537  20220611  
2    5493

oldcard2
      Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额  线路号  线路编号  \
0              4  9616900711049089  101  20220611203704   0.0  160    16   
1            181  9616900721950503    4  20220611203216   1.0   80     8   
2            182  9616900721653575  101  20220611203227   1.6   80     8   
3            183  9616900721543529  101  20220611203554   1.6   80     8   
4            378  9616900720327187  101  20220611205705   1.6  770    77   
...          ...               ...  ...             ...   ...  ...   ...   
1149       46511  9616900721511441  101  20220611204426   1.6  780    78   
1150       46536                 0   11  20220611203539   0.0  380    38   
1151       46537  9616900720405411    6  20220611203543   1.0  380    38   
1152       46538  9616900722041912   11  20220611203546   0.0  380    38   
1153       46550  9616900721791468  101  20220611203256   0.0  300    30   

          时间        日期  
0     123704  20220611  
1     123216  20220611  
2  

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611,53000,0,0,0
1,2209,9616900720307617,6,20220611141041,1.0,20,2,61041,20220611,53000,0,0,0
2,9428,9616900722227198,6,20220611153036,0.0,30,3,73036,20220611,53000,0,0,0
3,1241,0,11,20220611142155,0.0,50,5,62155,20220611,53000,0,0,0
4,41758,0,11,20220611150926,0.0,60,6,70926,20220611,53000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,8173,9616900730130399,6,20220611162433,0.5,3850,385,82433,20220611,153000,0,0,0
160,4351,9616900730079653,11,20220611145849,0.0,3870,387,65849,20220611,153000,0,0,0
161,43,9616900730044189,11,20220611142552,0.0,6120,612,62552,20220611,153000,1,1,2
162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611,153000,3,1,4


In [74]:
ALLIDSY4=ALLIDSY2
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
IDSY['段开始时间']=0
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0
for i in range(0,8):
    t1=920000+i*10000+3000
    t2=920000+i*10000+6000
    t3=40000+(i+12)*10000+3000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    ##按照线路名统计每条线上的刷卡数据
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t3
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
        #print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY4=ALLIDSY4.append(IDSY)      
ALLIDSY4

oldcard2
      Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0             15  9616900716032936  101  20220612005020   0.0   160    16   
1             16  9616900720164833  101  20220612005027   0.0   160    16   
2             38                 0   11  20220612003923   0.0   380    38   
3            292  9616900721707883    6  20220612003559   1.0  1070   107   
4            293  9616900720228852    6  20220612004148   1.0  1070   107   
...          ...               ...  ...             ...   ...   ...   ...   
1776       46601                 0   11  20220612005909   0.0   880    88   
1777       46602  9616900722002992   11  20220612005913   0.0   880    88   
1778       46603  9616900720047521   11  20220612005920   0.0   880    88   
1779       46604  9616900721722634   11  20220612005924   0.0   880    88   
1780       46658  9616900721774419    6  20220612003001   1.0  2030   203   

          时间        日期  
0     925020  20220611  
1     925027  20

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611,53000,0,0,0
1,2209,9616900720307617,6,20220611141041,1.0,20,2,61041,20220611,53000,0,0,0
2,9428,9616900722227198,6,20220611153036,0.0,30,3,73036,20220611,53000,0,0,0
3,1241,0,11,20220611142155,0.0,50,5,62155,20220611,53000,0,0,0
4,41758,0,11,20220611150926,0.0,60,6,70926,20220611,53000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,8173,9616900730130399,6,20220611162433,0.5,3850,385,82433,20220611,233000,0,0,0
160,4351,9616900730079653,11,20220611145849,0.0,3870,387,65849,20220611,233000,0,0,0
161,43,9616900730044189,11,20220611142552,0.0,6120,612,62552,20220611,233000,0,0,0
162,1162,9616900730121574,6,20220611124737,1.5,6220,622,44737,20220611,233000,0,0,0


In [75]:
ALLIDSY3.to_csv(r'F:\扬州\NEWALLIDSY3_11.csv')
ALLIDSY4.to_csv(r'F:\扬州\NEWALLIDSY4_11.csv')

In [76]:
ALL=ALLIDSY3
ALL=ALL.append(ALLIDSY4)
ALL.sort_values(by='段开始时间')

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3275,9616900721642849,11,20220611142655,0.0,11,1,62655,20220611,50000,0,0,0
58,1610,9616900721753777,101,20220611131126,1.6,710,71,51126,20220611,50000,1,1,2
57,344,9616900722241657,6,20220612003114,0.0,700,70,923114,20220611,50000,0,0,0
56,3998,9616900721644930,6,20220611213845,0.0,680,68,133845,20220611,50000,0,0,0
55,2091,9616900722183756,101,20220611141133,1.6,660,66,61133,20220611,50000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,2091,9616900722183756,101,20220611141133,1.6,660,66,61133,20220611,233000,0,0,0
56,3998,9616900721644930,6,20220611213845,0.0,680,68,133845,20220611,233000,0,0,0
57,344,9616900722241657,6,20220612003114,0.0,700,70,923114,20220611,233000,0,0,0
59,7768,0,101,20220611145534,1.6,720,72,65534,20220611,233000,0,0,0


In [77]:

ALL=ALL.loc[:,['段开始时间','线路编号','老年卡客运量','其他卡客运量','总客运量']]
ALL['30min开始时间']=0
for i in range(0,len(ALL)):
    T=ALL['段开始时间'].iloc[i]
    t=str(T)
    if len(t)==5:
        TS=t[-2:]
        TM=t[-4:-2]
        TH=t[0]
    if len(t)==6:
        TS=t[-2:]
        TM=t[-4:-2]
        TH=t[:2]
    t=TH+'：'+TM+'：'+TS
    ALL['30min开始时间'].iloc[i]=t

In [78]:
ALL

,段开始时间,线路编号,老年卡客运量,其他卡客运量,总客运量,30min开始时间
0,50000,1,0,0,0,5：00：00
1,50000,2,0,0,0,5：00：00
2,50000,3,0,0,0,5：00：00
3,50000,5,0,0,0,5：00：00
4,50000,6,0,0,0,5：00：00
...,...,...,...,...,...,...
159,233000,385,0,0,0,23：30：00
160,233000,387,0,0,0,23：30：00
161,233000,612,0,0,0,23：30：00
162,233000,622,0,0,0,23：30：00


In [80]:
ALL.to_excel(r'F:\扬州\30min刷卡统计_11(1).xlsx')